# Develop an LSTM Network Model

In [1]:
# INFO - example coming from:
# https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

In [2]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [3]:
# load a list of files into a 3D array of [samples, timesteps, features]
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [4]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [5]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

In [16]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 1, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.summary()
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [8]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [9]:
# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [17]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot
 

# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 100)               44000     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_26 (Dense)             (None, 6)                 606       
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
7352/7352 [==============================] - 20s 3ms/step - loss: 1.2577 - accuracy: 0.4395
Epoch 2/15
7352/7352 [======================

Epoch 1/15
7352/7352 [==============================] - 14s 2ms/step - loss: 1.3408 - accuracy: 0.4290
Epoch 2/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.8036 - accuracy: 0.6631
Epoch 3/15
7352/7352 [==============================] - 16s 2ms/step - loss: 0.5859 - accuracy: 0.7617
Epoch 4/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.5182 - accuracy: 0.8141
Epoch 5/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.4974 - accuracy: 0.8009
Epoch 6/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.3532 - accuracy: 0.8607
Epoch 7/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.2484 - accuracy: 0.9089
Epoch 8/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.2203 - accuracy: 0.9232
Epoch 9/15
7352/7352 [==============================] - 15s 2ms/step - loss: 0.1766 - accuracy: 0.9362
Epoch 10/15
7352/7352 [==============================] - 16s 2ms/step - l

7352/7352 [==============================] - 12s 2ms/step - loss: 0.2462 - accuracy: 0.9208
Epoch 10/15
7352/7352 [==============================] - 13s 2ms/step - loss: 0.1879 - accuracy: 0.9331
Epoch 11/15
7352/7352 [==============================] - 12s 2ms/step - loss: 0.1807 - accuracy: 0.9350
Epoch 12/15
7352/7352 [==============================] - 12s 2ms/step - loss: 0.1608 - accuracy: 0.9381
Epoch 13/15
7352/7352 [==============================] - 13s 2ms/step - loss: 0.1906 - accuracy: 0.9272
Epoch 14/15
7352/7352 [==============================] - 12s 2ms/step - loss: 0.1538 - accuracy: 0.9412
Epoch 15/15
7352/7352 [==============================] - 12s 2ms/step - loss: 0.1521 - accuracy: 0.9448
>#7: 91.415
Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 100)               44000     
___________________________________________________

# Develop a CNN-LSTM Network Model

In [18]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
	model.add(TimeDistributed(Dropout(0.5)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(LSTM(100))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.summary()
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [19]:
# cnn lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, None, 30, 64)      1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 28, 64)      12352     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 28, 64)      0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 14, 64)      0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 896)         0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 100)               398800    
___________________

>#6: 89.040
Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_31 (TimeDis (None, None, 30, 64)      1792      
_________________________________________________________________
time_distributed_32 (TimeDis (None, None, 28, 64)      12352     
_________________________________________________________________
time_distributed_33 (TimeDis (None, None, 28, 64)      0         
_________________________________________________________________
time_distributed_34 (TimeDis (None, None, 14, 64)      0         
_________________________________________________________________
time_distributed_35 (TimeDis (None, None, 896)         0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 100)               398800    
_________________________________________________________________
dropout_36 (Dropout)         (None, 100) 

# Develop a ConvLSTM Network Model

In [20]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape into subsequences (samples, time steps, rows, cols, channels)
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
	# define model
	model = Sequential()
	model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.summary()
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [22]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

# run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 1, 30, 64)         56320     
_________________________________________________________________
dropout_43 (Dropout)         (None, 1, 30, 64)         0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_65 (Dense)             (None, 100)               192100    
_________________________________________________________________
dense_66 (Dense)             (None, 6)                 606       
Total params: 249,026
Trainable params: 249,026
Non-trainable params: 0
_________________________________________________________________
>#1: 86.800
M

>#8: 89.752
Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_9 (ConvLSTM2D)  (None, 1, 30, 64)         56320     
_________________________________________________________________
dropout_51 (Dropout)         (None, 1, 30, 64)         0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_81 (Dense)             (None, 100)               192100    
_________________________________________________________________
dense_82 (Dense)             (None, 6)                 606       
Total params: 249,026
Trainable params: 249,026
Non-trainable params: 0
_________________________________________________________________
>#9: 91.551
Model: "sequential_43"
_________________________________________________________________


# Ejercicio 1 - Aplica la tarea de clasificacion de serie de datos a otra aplicacion

In [ ]:
# Se recomienda:
# Busca o generate una serie de datos

# Analiza los datos de entrada del ejemplo

# Generate una funcion de entrada para cargar los datos de entrenamiento

# Generate una funcion de entrada para cargar datos de test

# Cambia el formato de los datos para que tengan el mismo formato que los datos del ejemplo

# Entrena el ejemplo 10 veces, graba los 10 modelos y quedate con el mejor

# Utiliza el modelo para estimar la serie de datos


# BONUS - Aplica la tarea de clasificacion a otra aplicacion